# 지난 시간 결과
0.49정도의 결과를 가졌다
--> 정말 대충 짠 코드인데 절반을 맞췄다는게 신기하다

# Feedback
1) feature engineering
--> 어떤 feature들이 주요한 역할을 하는가?

2) 데이터 정규화
--> min-max, stardarization

3) MLP 
--> 다른 알고리즘들은 어떤게 있을까?
--> 파이토치로 구성해보는 건 어떨까?

4) threshold
--> 1.5를 기준으로해서 잡았는데, 이것은 무엇을 기준으로 잡을 수 있을까?


# Step1: Feature engineering
1) object변수들을 원핫인코딩으로 바꿔주겠다.
<br>
2) 좋은 Feature이 무엇인지 search해보자<br>
<i style ="border: 1px solid black;"> Feature Engineering이란 머신러닝 문제에 대해서 타겟에 대해서 최고로 잘 표현할 수 있게 가공되지 않은 데이터를 가공하는 것을 의미한다. <i>
<br>
1. You can isolate and highlight key information, which helps your algorithms “focus” on what’s important.
중요하다고 생각하는 변수를 강조시킬 수 있다.

2. You can bring in your own domain of expertise.
자신만의 도메인을 도입할 수 있다.

3. Most importantly, once you understand the “vocabulary” of feature engineering, you can bring in other people’s domain expertise!
<br>https://data-newbie.tistory.com/93

In [109]:
import pandas as pd
# 데이터 불러오기
train = pd.read_csv("/Users/igyuseog/Desktop/open data/train.csv")
test = pd.read_csv("/Users/igyuseog/Desktop/open data/test_x.csv")

# 첫번째 열에 해당하는 (index)를 지웠다.
train = train.iloc[:,1:]
test = test.iloc[:,1:]

#출력
train.head()
test.head()

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,3.0,736,2.0,2941,3.0,4621,1.0,4857,2.0,2550,...,0,1,0,0,1,0,1,0,1,1
1,3.0,514,2.0,1952,3.0,1552,3.0,821,4.0,1150,...,0,0,0,0,0,0,0,0,0,0
2,3.0,500,2.0,2507,4.0,480,2.0,614,2.0,1326,...,0,1,0,1,1,0,1,0,1,1
3,1.0,669,1.0,1050,5.0,1435,2.0,2252,5.0,2533,...,1,1,1,1,1,1,1,1,1,1
4,2.0,499,1.0,1243,5.0,845,2.0,1666,2.0,925,...,1,1,0,1,1,0,1,1,1,1


In [110]:
#object인 column명을 알 수 있다.
object_col = train.columns[train.dtypes == "object"].tolist()

#groupby (다양한 기능들이 존재하는데, 내가 원했던건 그룹별로 voted의 양상이었다.)
train.groupby(["gender"])["voted"].value_counts()


# --------------------------------------------------------

#step1) drop --> 필요없다고 판단되는 것들을 지웠다.
y = train["voted"]
train = train.drop(["engnat","hand","race","religion","urban","voted"], axis = 1)
test = test.drop(["engnat","hand","race","religion","urban"], axis = 1)

#step2)one_hot encoding은 아래와 같이 get_dumpy를 통해 가능하다.
train = pd.get_dummies(train)
test = pd.get_dummies(test)

#step3) 정규화 min-max
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
train = min_max_scaler.fit_transform(train)
test = min_max_scaler.fit_transform(test)

# train & test

In [83]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes = [10,10,10,10,10]).fit(train,y)

In [105]:
import numpy as np

pred = model.predict(train)
pred = np.where(pred < 1.483,1,2)

acc = sum(pred == y) / len(pred)
print("accuracy = ", acc)

accuracy =  0.6836730211719231


In [112]:
submit = pd.read_csv("/Users/igyuseog/Desktop/open data/sample_submission.csv")
pred = model.predict(test)
pred = np.where(pred < 1.483,1,2)

submit["voted"] = pred

In [123]:
submit.loc[:,"voted"].to_csv("/Users/igyuseog/Desktop/open data/MLP2.csv")